In [123]:
import collections
from gurobipy import *

In [124]:
n_cargos = 0
n_barcos = 0
n_puertos = 0
Barco = collections.namedtuple('Barco', ['puerto_origen', 'tiempo_salida', 'capacidad'])
barcos_dict = {}
compatibilidad = {}
Cargo = collections.namedtuple('Cargo', ['puerto_origen', 'puerto_destino', 'tamaño', 'SPOT', 'LT_carga', 'RT_carga', 'LT_descarga', 'RT_descarga'])
cargos_dict = {}

Viaje = collections.namedtuple('Viaje', ['tiempo_viaje', 'costo_viaje'])
viajes_dict = {}
Costos = collections.namedtuple('Costo', ['tiempo_origen', 'costo_origen', 'tiempo_destino', 'costo_destino'])
costos_dict = {}

In [125]:
with open("SHORTSEA_FUN_C8_V3_HE_1.txt", "r") as file:
    counter = 0
    while(True):
        line = file.readline()
        if line[0] == "#":
            counter +=1
        if counter == 6:
            n_cargos = int(line[21])
        if counter == 13:
            n_puertos = int(file.readline())
        if counter == 14:
            n_barcos = int(file.readline())
        if counter == 15:
            for i in range(n_barcos):
                index, home_port, starting_time, capacity = file.readline().strip().split(',')
                barcos_dict[int(index)] = Barco(int(home_port), int(starting_time), int(capacity))
        if counter == 17:
            for i in range(n_barcos):
                aux = file.readline().strip().split(',')
                index = aux[0]
                cargos_compatibles = [int(i) for i in aux[1:]]
                compatibilidad[int(index)] = cargos_compatibles
        if counter == 18:
            for i in range(n_cargos):
                index, puerto_origen, puerto_destino, tamano, SPOT, LT_carga, RT_carga, LT_descarga, RT_descarga= file.readline().strip().split(',')
                cargos_dict[int(index)] = Cargo(int(puerto_origen), int(puerto_destino), int(tamano), int(SPOT), int(LT_carga), int(RT_carga), int(LT_descarga), int(RT_descarga))
        if counter == 19:
            for i in range(n_puertos*n_puertos*n_barcos):
                index, puerto_origen, puerto_destino, tiempo_viaje, costo_viaje = file.readline().strip().split(',')
                viajes_dict[int(puerto_origen), int(puerto_destino),int(index)] = Viaje(int(tiempo_viaje), int(costo_viaje))
        if counter == 20:
            for i in range(n_barcos*n_cargos):
                vessel, cargo, tiempo_origen, costo_origen, tiempo_destino, costo_destino = file.readline().strip().split(',')
                costos_dict[int(vessel),int(cargo)] = Costos(int(tiempo_origen), int(costo_origen), int(tiempo_destino), int(costo_destino))
            break
        


In [126]:
print(n_cargos)
print(n_barcos)
print(n_puertos)
print(barcos_dict[1])
print(compatibilidad)
print(cargos_dict[1])
print(viajes_dict[10, 11, 3])
print(costos_dict[1,1].tiempo_origen)

8
3
39
Barco(puerto_origen=8, tiempo_salida=0, capacidad=100)
{1: [1, 2, 3, 4, 5, 7, 8], 2: [1, 2, 3, 4, 5, 6, 7, 8], 3: [1, 2, 3, 4, 5, 6, 7, 8]}
Cargo(puerto_origen=17, puerto_destino=37, tamaño=100, SPOT=866063, LT_carga=328, RT_carga=400, LT_descarga=328, RT_descarga=962)
Viaje(tiempo_viaje=8, costo_viaje=4694)
31


In [127]:
K= {} # conjunto capacidades de barcos
for i in barcos_dict.keys():
    K[i]= barcos_dict[i].capacidad
K

{1: 100, 2: 100, 3: 100}

In [128]:
# nodos de carga y descarga
NP = [i for i in cargos_dict.keys()]
ND = [i+n_cargos for i in cargos_dict.keys()]

In [129]:
nodos = {}
for i in cargos_dict.keys():
    nodos[i] = {'cargo': i, 'puerto': cargos_dict[i].puerto_origen , 'tamaño':cargos_dict[i].tamaño, 'SPOT':cargos_dict[i].SPOT, 'LT':cargos_dict[i].LT_carga, 'RT':cargos_dict[i].RT_carga}
    nodos[i+n_cargos] = {'cargo': i,'puerto': cargos_dict[i].puerto_destino , 'tamaño':cargos_dict[i].tamaño, 'SPOT':cargos_dict[i].SPOT, 'LT':cargos_dict[i].LT_descarga, 'RT':cargos_dict[i].RT_descarga}

In [130]:
o = {}
d = {}
for i in barcos_dict.keys():
    o[i] = i+n_cargos*2
    d[i] = i+n_barcos+n_cargos*2
    nodos[i+n_cargos*2] = {'cargo': 0, 'puerto': barcos_dict[i].puerto_origen , 'tamaño':0, 'SPOT':0, 'LT':barcos_dict[i].tiempo_salida, 'RT':barcos_dict[i].tiempo_salida}
    nodos[i+n_barcos+n_cargos*2] = {'cargo': 0,'puerto': 0 , 'tamaño':0, 'SPOT':0, 'LT':1253, 'RT':1253}

In [131]:
nodos

{1: {'cargo': 1,
  'puerto': 17,
  'tamaño': 100,
  'SPOT': 866063,
  'LT': 328,
  'RT': 400},
 9: {'cargo': 1,
  'puerto': 37,
  'tamaño': 100,
  'SPOT': 866063,
  'LT': 328,
  'RT': 962},
 2: {'cargo': 2,
  'puerto': 33,
  'tamaño': 100,
  'SPOT': 383620,
  'LT': 513,
  'RT': 585},
 10: {'cargo': 2,
  'puerto': 36,
  'tamaño': 100,
  'SPOT': 383620,
  'LT': 513,
  'RT': 960},
 3: {'cargo': 3,
  'puerto': 17,
  'tamaño': 100,
  'SPOT': 428848,
  'LT': 142,
  'RT': 214},
 11: {'cargo': 3,
  'puerto': 27,
  'tamaño': 100,
  'SPOT': 428848,
  'LT': 142,
  'RT': 547},
 4: {'cargo': 4,
  'puerto': 6,
  'tamaño': 100,
  'SPOT': 349124,
  'LT': 118,
  'RT': 190},
 12: {'cargo': 4,
  'puerto': 1,
  'tamaño': 100,
  'SPOT': 349124,
  'LT': 118,
  'RT': 557},
 5: {'cargo': 5,
  'puerto': 38,
  'tamaño': 100,
  'SPOT': 511606,
  'LT': 726,
  'RT': 798},
 13: {'cargo': 5,
  'puerto': 33,
  'tamaño': 100,
  'SPOT': 511606,
  'LT': 726,
  'RT': 1253},
 6: {'cargo': 6,
  'puerto': 10,
  'tamaño': 10

In [132]:
N = {}
for i in barcos_dict.keys():
    N[i] = []
    for j in compatibilidad[i]:
        N[i].append(j)
        N[i].append(j+n_cargos)
    N[i].append(o[i])
    N[i].append(d[i])
    N[i].sort()
N

{1: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 20],
 2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 21],
 3: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 22]}

In [133]:
Av={}
V=list(barcos_dict.keys())
indices = []
for v in V:
    Av[v] = [] 
    for i in N[v]:
        for j in N[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 15), (1, 16), (1, 17), (1, 20), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 15), (2, 16), (2, 17), (2, 20), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 15), (3, 16), (3, 17), (3, 20), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 15), (4, 16), (4, 17), (4, 20), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 15), (5, 16), (5, 17), (5, 20), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 15), (7, 16), (7, 17), (7, 20), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 15), (8, 16), (8, 17), (8, 20), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 7),

In [134]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in NP if i in N[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in ND if i in N[v]]

print(NP_v)
print(ND_v)

{1: [1, 2, 3, 4, 5, 7, 8], 2: [1, 2, 3, 4, 5, 6, 7, 8], 3: [1, 2, 3, 4, 5, 6, 7, 8]}
{1: [9, 10, 11, 12, 13, 15, 16], 2: [9, 10, 11, 12, 13, 14, 15, 16], 3: [9, 10, 11, 12, 13, 14, 15, 16]}


In [135]:
LT_i = {i:nodos[i]["LT"] for i in nodos.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:nodos[i]["RT"] for i in nodos.keys()} #upper bounds ventana de tiempo por nodo

print(LT_i)
print(RT_i)

{1: 328, 9: 328, 2: 513, 10: 513, 3: 142, 11: 142, 4: 118, 12: 118, 5: 726, 13: 726, 6: 0, 14: 0, 7: 21, 15: 21, 8: 63, 16: 63, 17: 0, 20: 1253, 18: 0, 21: 1253, 19: 0, 22: 1253}
{1: 400, 9: 962, 2: 585, 10: 960, 3: 214, 11: 547, 4: 190, 12: 557, 5: 798, 13: 1253, 6: 72, 14: 499, 7: 93, 15: 448, 8: 135, 16: 641, 17: 0, 20: 1253, 18: 0, 21: 1253, 19: 0, 22: 1253}


In [136]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    for j in nodos.keys():
        for v in V:
            cargo_i= nodos[i]['cargo']
            if i>n_cargos*2+n_barcos or j>n_cargos*2+n_barcos:
                T_ijv[i,j,v] = 0
            else: 
                puerto_origen = nodos[i]['puerto']
                puerto_destino = nodos[j]['puerto']
                T_ijv[i,j,v] = viajes_dict[i,j,v].tiempo_viaje
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i,j,v] += costos_dict[v,cargo_i].tiempo_origen
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i,j,v] += costos_dict[v,cargo_i].tiempo_destino
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j


In [137]:
C_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    for j in nodos.keys():
        for v in V:
            cargo_i= nodos[i]['cargo']
            if i>n_cargos*2+n_barcos or j>n_cargos*2+n_barcos:
                C_ijv[i,j,v] = 0
            else: 
                puerto_origen = nodos[i]['puerto']
                puerto_destino = nodos[j]['puerto']
                C_ijv[i,j,v] = viajes_dict[i,j,v].costo_viaje
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    C_ijv[i,j,v] += costos_dict[v,cargo_i].costo_origen
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i,j,v] += costos_dict[v,cargo_i].costo_destino

In [138]:
Qj={i:nodos[i]["tamaño"] for i in nodos.keys()}
print("Qj: ",Qj)

print("\n")

Cs = {i:nodos[i]["SPOT"] for i in nodos.keys()}
print("Cs", Cs)

Qj:  {1: 100, 9: 100, 2: 100, 10: 100, 3: 100, 11: 100, 4: 100, 12: 100, 5: 100, 13: 100, 6: 100, 14: 100, 7: 100, 15: 100, 8: 100, 16: 100, 17: 0, 20: 0, 18: 0, 21: 0, 19: 0, 22: 0}


Cs {1: 866063, 9: 866063, 2: 383620, 10: 383620, 3: 428848, 11: 428848, 4: 349124, 12: 349124, 5: 511606, 13: 511606, 6: 529259, 14: 529259, 7: 261382, 15: 261382, 8: 624544, 16: 624544, 17: 0, 20: 0, 18: 0, 21: 0, 19: 0, 22: 0}


In [139]:
aux = []
for i in V:
    for j in N[i]:
        aux.append((j,i))
aux = tuplelist(aux)

In [144]:
nodos_index = list(nodos.keys())
nodos_index

[1, 9, 2, 10, 3, 11, 4, 12, 5, 13, 6, 14, 7, 15, 8, 16, 17, 20, 18, 21, 19, 22]

In [150]:

# Model
m = Model("Transporte Maritimo")

# variables

x = m.addVars(nodos_index, nodos_index, V, vtype=GRB.BINARY, name="x")
y = m.addVars(NP,vtype=GRB.BINARY,name="y")
t = m.addVars(aux,vtype=GRB.INTEGER, name="t")
l = m.addVars(aux,vtype=GRB.INTEGER,name="l")

In [151]:
# Objective function
m.setObjective(sum(sum(C_ijv[i,j,v]*x[i,j,v] for (i,j) in Av[v]) for v in V)+sum(Cs[i]*y[i] for i in NP), GRB.MINIMIZE)

In [152]:
c1 = m.addConstrs(((sum(sum(x[i,j,v]  for v in V) for j in N[v])  + y[i]== 1) for i in NP), name = "Const_1")

In [153]:
c2 = m.addConstrs(((sum(x[o[v],j,v] for j in N[v]) == 1) for v in V), name = "Const_2")

In [154]:
c3 = m.addConstrs((((sum(x[i,j,v] for j in N[v])-sum(x[j,i,v] for j in N[v]))==0) for v in V for i in N[v] if i!=o[v] or i!=o[v]), name = "Const_3")

In [155]:
c4 = m.addConstrs(((sum(x[j,d[v],v] for j in N[v]) == 1) for v in V), name = "Const_4")

In [156]:
m.addConstrs(((l[i,v]+Qj[j]-l[j,v]<=K[v]*(1-x[i,j,v])) for v in V  for j in NP_v[v] for i in N[v] if ((i,j) in Av[v])), name = "Const_5")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting M

In [157]:
c6 = m.addConstrs(((l[i,v]-Qj[j]-l[n_cargos+j,v]<=K[v]*(1-x[i,n_cargos+j,v]))  for v in V for j in NP_v[v] for i in N[v] if ((i,j+n_cargos) in Av[v])), name = "Const_6")

In [158]:
m.addConstrs(((l[i,v]<=K[v]) for v in V for i in NP_v[v]), name = "Const_7_ub")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,


In [159]:
m.addConstrs(((l[i,v]>=0) for v in V for i in NP_v[v]), name = "Const_7_lb")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,


In [160]:
c8 = m.addConstrs(((t[i,v]+T_ijv[i,j,v]-t[j,v]<=(RT_i[i]+T_ijv[i,j,v])*(1-x[i,j,v])) for v in V for (i,j) in Av[v]), name = "Const_8")

In [161]:
c9 = m.addConstrs(((sum(x[i,j,v] for j in N[v])-sum(x[n_cargos+i,j,v] for j in N[v])==0) for v in V for i in NP_v[v]), name = "Const_9")

In [162]:
c10 = m.addConstrs(((t[i,v]+T_ijv[i,n_cargos+i,v]-t[n_cargos+i,v]<=0) for v in V for i in NP_v[v] ), name = "Const_10")

In [163]:
c11ub = m.addConstrs(((t[i,v]<=RT_i[i]) for v in V for i in N[v]), name = "Const_11_ub")

In [164]:
c11lb = m.addConstrs(((LT_i[i]<=t[i,v]) for v in V for i in N[v]), name = "Const_11_lb")

In [165]:
# Solve
m.optimize()

valorobjetivo = m.objval

"""for v in m.getVars():
    print('%s %g' % (v.varName, v.x))"""

print('\n Valor óptimo: %8.4f \n' % valorobjetivo)

m.write("Transporte_final.sol")
m.write("model.lp")

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1963 rows, 1564 columns and 8056 nonzeros
Model fingerprint: 0x131d671c
Variable types: 0 continuous, 1564 integer (1460 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+04, 9e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

 Valor óptimo:   0.0000 

